1. Install and Import Dependencies

In [1]:
!pip3 install torch torchvision torchaudio


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

2. Instantiate Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

3. Encode and Calculate Sentiment

In [5]:
tokens = tokenizer.encode('I hate this, absolutely the worst', return_tensors='pt')

In [6]:
tokens[0]

tensor([  101,   151, 39487, 10372,   117, 35925, 10563, 10103, 43060,   102])

In [8]:
tokenizer.decode(tokens[0])

'[CLS] i hate this, absolutely the worst [SEP]'

In [9]:
result = model(tokens)

In [10]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 5.0607,  1.6029, -1.0123, -3.0154, -1.8469]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [11]:
result.logits

tensor([[ 5.0607,  1.6029, -1.0123, -3.0154, -1.8469]],
       grad_fn=<AddmmBackward0>)

In [12]:
int(torch.argmax(result.logits))+1

1

4. Collect Reviews

In [13]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup= BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [14]:
reviews[0]

'The food is fresh and tasty. \xa0The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.'

5. Load Reviews into DataFrame Score

In [15]:
import numpy as np
import pandas as pd

In [16]:
df = pd.DataFrame(np.array(reviews), columns= ['review'])

In [17]:
df.head()

,review
0,The food is fresh and tasty. The scallop cevi...
1,Don't come here expecting legit Mexican food b...
2,Out of all the restaurants that I tried in Syd...
3,We came here on a Thursday night @ 5pm and by ...
4,I was pleasantly surprised at what a great job...


In [18]:
df.tail()

,review
5,Have been here twice and have absolutely loved...
6,Really nice (upmarket) Mexican restaurant. Goo...
7,If you're looking for a quiet little romantic ...
8,The service at this place was top notch - the ...
9,Ordered feed me for $59 along with that.. Food...


In [19]:
df['review'].iloc[0]

'The food is fresh and tasty. \xa0The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.'

In [20]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors= 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [21]:
sentiment_score(df['review'].iloc[1])

3

In [22]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [23]:
df['review']

0    The food is fresh and tasty.  The scallop cevi...
1    Don't come here expecting legit Mexican food b...
2    Out of all the restaurants that I tried in Syd...
3    We came here on a Thursday night @ 5pm and by ...
4    I was pleasantly surprised at what a great job...
5    Have been here twice and have absolutely loved...
6    Really nice (upmarket) Mexican restaurant. Goo...
7    If you're looking for a quiet little romantic ...
8    The service at this place was top notch - the ...
9    Ordered feed me for $59 along with that.. Food...
Name: review, dtype: object

In [24]:
df

,review,sentiment
0,The food is fresh and tasty. The scallop cevi...,4
1,Don't come here expecting legit Mexican food b...,3
2,Out of all the restaurants that I tried in Syd...,5
3,We came here on a Thursday night @ 5pm and by ...,4
4,I was pleasantly surprised at what a great job...,5
5,Have been here twice and have absolutely loved...,5
6,Really nice (upmarket) Mexican restaurant. Goo...,4
7,If you're looking for a quiet little romantic ...,2
8,The service at this place was top notch - the ...,5
9,Ordered feed me for $59 along with that.. Food...,2


In [25]:
df['review'].iloc[3]

'We came here on a Thursday night @ 5pm and by 6pm the place was packed. A lovely big restaurant with a bar at the front (which is a bit awkward to try and push past everyone to get to your table). Friendly, helpful staff which is always a good start. The menu is large so we went with the "feed me" selection. All you need to do is sit back and let the chef feed you. As the other reviewers have stated the corn is a highlight and the pulled pork tacos, the sangria wasn\'t bad either.Loved the Mexican tapas style food and will be back.'